# Map
Show which packages we used:

In [9]:
import pandas as pd
import folium
from geopy import distance
from folium.plugins import MarkerCluster

Importing data with **read_csv()** and we filter the columns with `usecols`.
#### Clean up the data:
- Remove all the duplicates from the data frame.
- Remove all the NaN values: this is an important step because if there are null values in the longitude and latitude columns we are not able to locate the house.
- If `df.average_rate_per_night`it is equal to zero, maybe there's a mistake because it's not possible that it's free! For this we delete this data from our analysis.

In [58]:
df = pd.read_csv('Airbnb_Texas_Rentals.csv',usecols=['latitude','longitude','average_rate_per_night','title','url'])
df = df.drop_duplicates()
df = df[df.average_rate_per_night != 0]
df = df.dropna()

In [3]:
df.head(2) # show the first 2 rows

,average_rate_per_night,latitude,longitude,title,url
0,$27,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...
1,$149,29.503068,-98.447688,Unique Location! Alamo Heights - Designer Insp...,https://www.airbnb.com/rooms/17481455?location...


Takes in input a set of coordinates and a maximum distance from the coordinates. For example, 
we can take the coordinates of *Dallas* (32.892048, - 96.871741) but, obviously,the customer can enter the coordinates related to the city where he wants to stay in Texas!

In [5]:
latitude = float(input('Enter a latitude: ')) #32.892048 
longitude = float(input('Enter a longitude: ')) #-96.871741
dist = float(input('Enter a distance (km): ')) #2

Enter a latitude: 32.892048
Enter a longitude: -96.871741
Enter a distance (km): 2


### It's time to create our map!

The first thing to do is to create a map object centered around the area of interest. Then, we added in each of the points using a for loop on the data frame df. For each location, we also create a popup tooltip to provide the verage_rate_per_night and if you click on the price you can go directly to the site of the house.

We noticed that several points are hard to see since they can be very close to each other. To fix these issues we created **marker clusters**: *the marker clusers group points that overlap and then it labels the resulting cirlce with the number of points in that area. If you click on the circle, the map zooms to the area to show you the individual points*.

In [53]:
m = folium.Map(location = [latitude, longitude], tiles ="OpenStreetMap", zoom_start = 10)
folium.Marker(location = [latitude,longitude],
              popup = 'Town center', icon = folium.Icon(color = 'green', icon = 'home')).add_to(m)
folium.Circle(location = [latitude, longitude], 
              radius = dist*1000,color='#3186cc',fill_color = '#000000',fill_opacity= 0.2).add_to(m)
mc = MarkerCluster()
for row in df.itertuples():
    k = distance.distance((latitude, longitude), (row.latitude, row.longitude)).km
    if k <= dist:
         mc.add_child(folium.Marker(location = [row.latitude, row.longitude],
                      popup = folium.Popup("<a href=" + row.url + ">" + row.average_rate_per_night + " </a>"),
                      icon = folium.Icon(color = 'lightgray', icon_color = '#3186cc')))
m.add_child(mc)
m.save('Map_result.html')